In [2]:
import numpy as np 
import pandas as pd
import cplex # community edition
from scipy.optimize import minimize

In [3]:
s1 = pd.read_csv('D:/jupyter_dir/yancao/data/step1.csv', dtype={'品规ID':str})
s2 = pd.read_csv('D:/jupyter_dir/yancao/data/step2.csv', dtype={'品规ID':str})
s2.head()

,品规ID,档位,订足面,订足率,订单满足率,订购率,上期策略,总户数,订购量,需求量,投放量,品规名
0,6901028010290,二十九档,0.500000,0.333333,0.500000,0.009091,2.0,660.0,4.0,8.0,12.0,中南海(京韵细支烤烟)
1,6901028010290,二十七档,0.142857,inf,0.222222,0.020349,0.0,344.0,2.0,9.0,0.0,中南海(京韵细支烤烟)
2,6901028010290,二十档,0.000000,NaN,0.000000,0.002648,0.0,1133.0,0.0,3.0,0.0,中南海(京韵细支烤烟)
3,6901028010290,二十八档,0.000000,0.000000,0.000000,0.009202,1.0,326.0,0.0,3.0,3.0,中南海(京韵细支烤烟)
4,6901028010290,十四档,0.000000,NaN,0.000000,0.000934,0.0,1071.0,0.0,13.0,0.0,中南海(京韵细支烤烟)


In [105]:
arg_source = pd.read_csv('D:/jupyter_dir/yancao/data/step1.csv')
arg_source.head()

# 参数和决策变量
price = arg_source['单箱价格'].values
sold = arg_source['上期销售量'].values 
cap = arg_source['产能'].values

sold_obj = 2300   # 目标销量(箱)
price_obj = 400000 # 目标结构(元/箱)
C = 10000


$
min f = (250 * Sold\_obj - \sum_{i=1}^I Sold\_next_i)^2 + C*\sum_{i=1}^I [(Sold\_next_i - Sold_i)/Sold_i]^2
$

$s.t.$

$
0 <= Sold\_next_i <= Cap_i
$

$
Sold\_obj*Price\_obj*0.9 <= \sum_{i=1}^I Price_i*Sold\_next_i/250 <= Sold\_obj*Price\_obj*1.1
$

目标函数拆成三项,去掉常数项

$
-500*sold\_obj*(x_1+x_2+...+x_n)
\\
[2x_0^2+2x_1^2+...+2x_n^2  +  4x_1x_2+4x_1x_3+...+4x_{n-1}x_n]/2 
\\
[2C/sold_0^2 * x_0^2 + 2C/sold_1^2 * x_1^2 + ... + 2C/sold_n^2 * x_n^2]/2 - (2C/sold_0*x_0 + 2C/sold_1*x_1 +...+2C/sold_n*x_n)
$

1、4项合并，2、3项合并，只保留一个二次项\[$x_1x_2$, $x_1^2$等\]/2即可

此处二次项LP文件格式参考：
https://www.ibm.com/support/knowledgecenter/zh/SSSA5P_12.7.0/ilog.odms.cplex.help/CPLEX/FileFormats/topics/LP_Quadratic.html


In [106]:
# lp文件生成
# sold_next --> x(0) x(1) ...

var = [i for i in map(lambda x: 'x(' + str(x) + ')', list(range(arg_source.shape[0])))] # ['x(1)', 'x(2)', ...]
price_sold_sum = ''
for i in range(len(cap)):
    price_sold_sum += ' + %s x(%s)'%(price[i]/250, i)

lp = "Minimize\n obj: "
for i in range(len(cap)):
    lp += ' - %s x(%s)'%(500*sold_obj+2*C/sold[i], i)

lp += '\n+ ['
for i in range(len(cap)):
    lp += ' + %s x(%s)^2'%(2*C/sold[i]**2 + 2, i)
    for j in range(i+1, len(cap)):
        lp += ' + 4 x(%s) * x(%s)'%(i,j)
    lp += '\n'

lp += ']/2\n'

lp += ' + %s'%(sold_obj**2 + C*len(cap))

lp += '\nSubject To\n'
lp += ' c1: %s >= %s \n'%(price_sold_sum, 0.9*sold_obj*price_obj)
lp += ' c2: %s <= %s \n'%(price_sold_sum, 1.1*sold_obj*price_obj)
for i in range(len(cap)):
    lp += ' c%s: x(%s) - Rgc%s = 0\n'%(3+i, i, i)
lp += 'Bounds\n'
for i in range(len(cap)): 
    lp += 'x(%s) Free\n'%i
for i in range(len(cap)): 
    lp += ' 0 <= Rgc%s <= %s \n'%(i, cap[i])
lp += 'General\n'
for i in range(len(cap)): lp += ' %s'%(var[i])
lp += '\nEnd'

with open('write.lp', 'w') as f:
    f.write(lp)
# print(lp)

In [108]:
my_prob = cplex.Cplex('write.lp')
my_prob.solve()

# print("\nSolution status = ", my_prob.solution.get_status(), ":", end=' ')
# print(my_prob.solution.status[my_prob.solution.get_status()])
# print("Solution value  = ", my_prob.solution.get_objective_value())

# slack = my_prob.solution.get_linear_slacks()
x = my_prob.solution.get_values()

np.array(x, dtype='int')[0:int(len(x)/2)]

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Row 'c1' infeasible, all entries at implied bounds.
Presolve time = 0.00 sec. (0.08 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.09 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.09 ticks)
CPLEX Error  1217: No solution exists.


CplexSolverError: CPLEX Error  1217: No solution exists.

In [181]:
len(x)

142

In [174]:
obj = lambda x: ((sold_obj*250 - x.sum())**2)*C + (((x - sold)/sold)**2).sum()

# 约束条件
cons = []
cons.append({'type':'ineq', 'fun': lambda x: (x*price).sum() - sold_obj*price_obj*0.9})
cons.append({'type':'ineq', 'fun': lambda x: sold_obj*price_obj*1.1 - (x*price).sum()})
for i in range(sold.size):
    cons.append({'type':'ineq', 'fun': lambda x: x[i]})
    cons.append({'type':'ineq', 'fun': lambda x: cap[i] - x[i]})

# 求解
sold_next = sold # 初始化决策变量
sol = minimize(obj, sold_next, method='TNC', constraints=cons)
np.array(sol.x, dtype=int).size

71

In [159]:
arg_source['sold_next'] = pd.Series(np.round(x), dtype='int')
arg_source

,品规名,单箱价格,品规ID,上期销售量,产能,sold_next
0,中南海(京韵细支烤烟),75000,6901028010290,14,42,14
1,云烟(中支金腰带),75000,6901028340670,183,243,183
2,云烟(细支大重九),250000,6901028312356,153,186,153
3,云烟(细支珍品),65000,6901028339537,474,580,474
4,云烟(软大重九),245000,6901028051989,80,122,80
5,云烟(软珍品),60000,6901028045919,4695,5643,4729
6,冬虫夏草(和润),125000,6901028177214,522,657,522
7,凤凰(细支),87500,6901028018166,594,728,594
8,利群(蓝天),40000,6901028207089,8018,9654,8117
9,利群(西子阳光),77500,6901028119009,714,870,714


In [201]:
arg_source['单箱价格'] # 25000~250000

250000